# Prediction

##Read and merge Data

In [ ]:
statusdf.count()

In [ ]:
stationdf.count()

In [ ]:
masterdf = statusdf.join(stationdf,statusdf.station_id ==  stationdf.id, 'inner')

In [ ]:
masterdf.show(5)

In [ ]:
masterdf = masterdf.drop('id','name', 'installation_date', 'dock_count', 'docks_available', 'station_id')

In [ ]:
masterdf.show(5)

In [ ]:
date_time_cols = split(masterdf['time'], ' ')

In [ ]:
masterdf = masterdf.withColumn('date', date_time_cols.getItem(0)).withColumn('time', date_time_cols.getItem(1))

In [ ]:
masterdf.show(5)

In [ ]:
time_cols = split(masterdf['time'], ':')
date_cols = split(masterdf['date'], '/')
masterdf = masterdf.withColumn('hour', time_cols.getItem(0).cast(IntegerType())).withColumn('minute', time_cols.getItem(1).cast(IntegerType())).withColumn('year', date_cols.getItem(0).cast(IntegerType())).withColumn('month', date_cols.getItem(1).cast(IntegerType())).withColumn('day', date_cols.getItem(2).cast(IntegerType()))

In [ ]:
masterdf = masterdf.drop('time','date')

In [ ]:
masterdf.show(5)

In [ ]:
#masterdf = masterdf.filter(masterdf.time.like("__:05:__")|masterdf.time.like("__:10:__"))
masterdf = masterdf.filter(masterdf.minute % 5 == 0)

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
# index = StringIndexer(inputCol="city", outputCol="cityEncoded")
# masterdf = index.fit(masterdf).transform(masterdf)
masterdf.show(5)

In [ ]:
df = masterdf

## Cleaning the Data

In [ ]:
# from pyspark.sql.functions import *
# df = masterdf.withColumn('name', regexp_replace('name', ',', ''))

In [ ]:
print((df.count(), len(df.columns)))

In [ ]:
# df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [ ]:
df_clean=df.dropna()
df_clean=df_clean.dropDuplicates()
df_clean.show()

+---------------+---------+-------------------+--------+----+------+----+-----+---+
|bikes_available|      lat|               long|    city|hour|minute|year|month|day|
+---------------+---------+-------------------+--------+----+------+----+-----+---+
|              2|37.329732|-121.90178200000001|San Jose|  14|    10|2013|    8| 29|
|              3|37.329732|-121.90178200000001|San Jose|   2|    45|2013|    8| 30|
|             11|37.329732|-121.90178200000001|San Jose|  18|    30|2013|    8| 30|
|             11|37.329732|-121.90178200000001|San Jose|   1|     0|2013|    9|  1|
|             11|37.329732|-121.90178200000001|San Jose|  10|    40|2013|    9|  1|
|              7|37.329732|-121.90178200000001|San Jose|  14|    10|2013|    9|  1|
|             10|37.329732|-121.90178200000001|San Jose|  23|    30|2013|    9|  2|
|             11|37.329732|-121.90178200000001|San Jose|  12|    35|2013|    9|  3|
|              9|37.329732|-121.90178200000001|San Jose|   0|    55|2013|   

In [ ]:
print((df_clean.count(), len(df_clean.columns)))

(3418966, 9)


In [ ]:
df_clean = df_clean \
    .withColumn("lat", df_clean["lat"].cast(FloatType()))   \
    .withColumn("bikes_available", df_clean["bikes_available"].cast(IntegerType()))   \
    .withColumn("long", df_clean["long"].cast(FloatType()))

df_clean.printSchema()

In [ ]:
df_clean.select(
    percentile_approx("bikes_available", [0.25, 0.5, 0.75], 1000000).alias("quantiles")).collect()

In [ ]:
q1_bikes = df_clean.select(percentile_approx("bikes_available", [0.25], 1000000).alias("quantiles")).collect()[0]['quantiles'][0]
q3_bikes = df_clean.select(percentile_approx("bikes_available", [0.75], 1000000).alias("quantiles")).collect()[0]['quantiles'][0]
iqr_bikes = q3_bikes - q1_bikes

#Check the Outlier Dock_counts
top_outlier_bikes = q3_bikes + 1.5 * iqr_bikes
bottom_outlier_bikes = q1_bikes - 1.5 * iqr_bikes
print(top_outlier_bikes, bottom_outlier_bikes)

In [ ]:
df_clean.show()

In [ ]:
df_selected = df_clean.select("bikes_available", "city", "lat", "long", "hour", "minute", "year", "month", "day")
df_selected.printSchema()

## Convert the Categorical Variables

In [ ]:
indexer = StringIndexer(inputCols=["city"],
                        outputCols=["cityIndex"],
                        stringOrderType="alphabetAsc")
indexed = indexer.fit(df_selected).transform(df_selected)
indexed.show()

In [ ]:
encoder = OneHotEncoder(inputCols=["cityIndex"],
                        outputCols=["categoryCity"])
model = encoder.fit(indexed)
encoded = model.transform(indexed)
encoded.show()

In [ ]:
encoded.columns

## Make the Features into a Vector

In [ ]:
featureassembler=VectorAssembler(inputCols=['categoryCity', 'lat', 'long', 'hour', 'minute', 'year', 'month', 'day'],
                                 outputCol="SelectFeatures")
output=featureassembler.transform(encoded)
output.show()

In [ ]:
finalized_data=output.select("SelectFeatures","bikes_available")


In [ ]:
##train test split
train_data,test_data=finalized_data.randomSplit([0.8,0.2])

## Simple Linear Regression

In [ ]:
regressor=LinearRegression(featuresCol='SelectFeatures', labelCol='bikes_available')
regressor=regressor.fit(train_data)
print("Coefficients: " + str(regressor.coefficients))
print("Intercept: " + str(regressor.intercept))

In [ ]:
trainingSummary = regressor.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

In [ ]:
### Prediction of Linear Regression
pred_results=regressor.evaluate(test_data)

In [ ]:
pred_results.meanAbsoluteError,pred_results.rootMeanSquaredError

## Random Forest Model

In [ ]:
rf = RandomForestRegressor(featuresCol='SelectFeatures', labelCol='bikes_available')
rf_model=rf.fit(train_data)

In [ ]:
### Prediction of random forest model

rf_predictions = rf_model.transform(test_data)
rf_evaluator = RegressionEvaluator(labelCol="bikes_available", predictionCol="prediction", metricName="rmse")
rmse = rf_evaluator.evaluate(rf_predictions)
rf_evaluator = RegressionEvaluator(labelCol="bikes_available", predictionCol="prediction", metricName="r2")
r2 = rf_evaluator.evaluate(rf_predictions)
rf_evaluator = RegressionEvaluator(labelCol="bikes_available", predictionCol="prediction", metricName="mae")
mae = rf_evaluator.evaluate(rf_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g\n(R2) on test data = %g\n(MAE) on test data = %g"%(rmse, r2, mae))
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print(" (R2) on test data = %g" % r2)
print(" (MAE) on test data = %g" % mae)


Root Mean Squared Error (RMSE) on test data = 3.30922
(R2) on test data = 0.15101
(MAE) on test data = 2.5114
Root Mean Squared Error (RMSE) on test data = 3.30922
 (R2) on test data = 0.15101
 (MAE) on test data = 2.5114


In [ ]:
rf_predictions.show()

In [ ]:
rf_model.featureImportances

In [ ]:
finalized_data.select('SelectFeatures').take(2)

## Gradient Boosting Tree Model

In [ ]:
train_data,test_data=finalized_data.randomSplit([0.8,0.2])
gbt=GBTRegressor(featuresCol='SelectFeatures', labelCol='bikes_available')
gbt_model=gbt.fit(train_data)

In [ ]:
### Prediction of GBT Model
gbt_predictions = gbt_model.transform(test_data)
# gbt_evaluator = RegressionEvaluator(labelCol="bikes_available", predictionCol="prediction", metricName="rmse")
# rmse = gbt_evaluator.evaluate(gbt_predictions)
gbt_evaluator = RegressionEvaluator(labelCol="bikes_available", predictionCol="prediction")
r2 = gbt_evaluator.evaluate(gbt_predictions, {gbt_evaluator.metricName: "r2"})
rmse = gbt_evaluator.evaluate(gbt_predictions, {gbt_evaluator.metricName: "rmse"})
mae = gbt_evaluator.evaluate(gbt_predictions, {gbt_evaluator.metricName: "mae"})

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("Mean Average Error (MAE) on test data = %g" % mae)
print("R2 (R2) on test data = %g" % r2)
